In [6]:
import tensorflow as tf
import os
import glob
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import EfficientNetV2B2
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout, Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import Callback

In [3]:
physical_devices = tf.config.list_physical_devices('GPU')
if len(physical_devices) == 0:
    raise RuntimeError("GPU tidak ditemukan.")

In [4]:
IMG_SIZE = 299
BATCH_SIZE = 32

In [5]:
image_paths = glob.glob('Data/food-101/images/*/*.jpg')
labels = [os.path.split(os.path.split(path)[0])[1] for path in image_paths]
unique_labels = sorted(set(labels))
label_to_index = {label: index for index, label in enumerate(unique_labels)}
labels = [label_to_index[label] for label in labels]

def load_and_preprocess(path, label):
    image = tf.io.read_file(path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, [IMG_SIZE, IMG_SIZE])
    return image, label

train_paths, test_paths, train_labels, test_labels = train_test_split(image_paths, labels, test_size=0.2, random_state=42)
train_data = tf.data.Dataset.from_tensor_slices((train_paths, train_labels)).map(load_and_preprocess).batch(BATCH_SIZE)
test_data = tf.data.Dataset.from_tensor_slices((test_paths, test_labels)).map(load_and_preprocess).batch(BATCH_SIZE)

In [7]:
input = Input(shape=(IMG_SIZE, IMG_SIZE, 3))
base_model = EfficientNetV2B2(include_top=False, weights='imagenet', input_shape=(IMG_SIZE, IMG_SIZE, 3))
base_model.trainable = False

x = base_model(input)

x = GlobalAveragePooling2D()(x)

predictions = Dense(101, activation='softmax')(x)

model = Model(inputs=input, outputs=predictions)

model.compile(optimizer=Adam(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

35839040/35839040 [==============================] - 56s 2us/step


In [8]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 299, 299, 3)]     0         
                                                                 
 efficientnetv2-b2 (Function  (None, 10, 10, 1408)     8769374   
 al)                                                             
                                                                 
 global_average_pooling2d (G  (None, 1408)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 101)               142309    
                                                                 
Total params: 8,911,683
Trainable params: 142,309
Non-trainable params: 8,769,374
_________________________________________________________________


In [9]:
history = model.fit(train_data, epochs=4, validation_data=test_data)

Epoch 1/4
2525/2525 [==============================] - 620s 235ms/step - loss: 1.7080 - accuracy: 0.5787 - val_loss: 1.2563 - val_accuracy: 0.6765
Epoch 2/4
2525/2525 [==============================] - 589s 233ms/step - loss: 1.2324 - accuracy: 0.6756 - val_loss: 1.1446 - val_accuracy: 0.7021
Epoch 3/4
2525/2525 [==============================] - 591s 234ms/step - loss: 1.1155 - accuracy: 0.7056 - val_loss: 1.0948 - val_accuracy: 0.7168
Epoch 4/4
2525/2525 [==============================] - 594s 235ms/step - loss: 1.0419 - accuracy: 0.7229 - val_loss: 1.0638 - val_accuracy: 0.7246


In [11]:
base_model.trainable = True
for layer in base_model.layers[:-30]:
    layer.trainable = False

model.compile(optimizer=Adam(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

class MyCallback(Callback):
    def on_epoch_end(self, epoch, logs=None):
        if logs is None:
            logs = {}
        if logs.get('val_accuracy') > 0.80:
            print("\nAkurasi telah mencapai 80%, menghentikan pelatihan.")
            self.model.stop_training = True

callback = MyCallback()

In [12]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 299, 299, 3)]     0         
                                                                 
 efficientnetv2-b2 (Function  (None, 10, 10, 1408)     8769374   
 al)                                                             
                                                                 
 global_average_pooling2d (G  (None, 1408)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 101)               142309    
                                                                 
Total params: 8,911,683
Trainable params: 1,509,709
Non-trainable params: 7,401,974
_________________________________________________________________


In [13]:
history = model.fit(train_data, epochs=4, validation_data=test_data, callbacks=callback)

Epoch 1/4
2525/2525 [==============================] - 670s 259ms/step - loss: 1.0103 - accuracy: 0.7284 - val_loss: 0.8724 - val_accuracy: 0.7679
Epoch 2/4
2525/2525 [==============================] - 668s 264ms/step - loss: 0.7613 - accuracy: 0.7913 - val_loss: 0.8274 - val_accuracy: 0.7810
Epoch 3/4
2525/2525 [==============================] - 682s 270ms/step - loss: 0.6236 - accuracy: 0.8281 - val_loss: 0.8216 - val_accuracy: 0.7851
Epoch 4/4
2525/2525 [==============================] - 681s 270ms/step - loss: 0.5136 - accuracy: 0.8547 - val_loss: 0.8236 - val_accuracy: 0.7881


In [14]:
model.save("model.h5")